## 8.1 Transfer Learning Faster R-CNN

⚠️⚠️⚠️ *Please open this notebook in Google Colab* by click below link ⚠️⚠️⚠️<br><br>
<a href="https://colab.research.google.com/github/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%208/8.1%20transfer_learning_faster_rcnn.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><br><br><br>
- Click `Connect` button in top right Google Colab notebook,<br>
<img src="https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%207/resource/cl-connect-gpu.png?raw=1" width="250px">
- If connecting process completed, it will turn to something look like this<br>
<img src="https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%207/resource/cl-connect-gpu-success.png?raw=1" width="250px">

- Check GPU connected into Colab environment is active

In [ ]:
!nvidia-smi

- Install necasarry package

In [ ]:
!pip install torchmetrics
!pip install onnx

#### 8.1.1 Download Dataset From Roboflow
- Folow instruction in [4.1 dataset_annotation_roboflow.ipynb](https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%204/4.1%20dataset_annotation_roboflow.ipynb) to prepare `Scissors Dataset` example,
- Open `Roboflow` > `Project` > `Versions` menu
- Then click `Download Dataset`<br>
<img src="https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%208/resource/rb-download-dataset.png?raw=1" width="850px">
- Choose `COCO` format and select `Show download code` then click `Continue` <br>
<img src="https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%208/resource/rb-download-format.png?raw=1" width="350px">
- click `Copy` icon to copy roboflow download code<br>
<img src="https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%208/resource/rb-copy-download-code.png?raw=1" width="350px">
- Then <font color="orange">replace below code</font> using the copied roboflow download code above,


In [ ]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="xxxxxxxxxxx")
# project = rf.workspace("xxxxxxx").project("xxxxxxxxxxxx")
# version = project.version(1)
# dataset = version.download("coco")

In [ ]:
import torch
import torch.optim as optim
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchmetrics.detection.mean_ap import MeanAveragePrecision

import cv2
import json
import os

- Create Pytorch Custom Dataset Loader to load `Scissors Dataset` from Roboflow

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.root = root

        # Load COCO JSON annotations
        with open(os.path.join(root, "_annotations.coco.json")) as f:
            data = json.load(f)

        # Process images and annotations
        self.images = {img['id']: img for img in data['images'] if img['file_name'].endswith('.jpg')}
        self.annotations = data['annotations']
        self.categories = {cat['id']: cat['name'] for cat in data['categories']}

        # Group annotations by image ID for easy lookup
        self.img_to_anns = {img_id: [] for img_id in self.images.keys()}
        for ann in self.annotations:
            self.img_to_anns[ann['image_id']].append(ann)

        # Store only images that have annotations
        self.imgs = [img_id for img_id in self.img_to_anns if self.img_to_anns[img_id]]

    def __getitem__(self, idx):
        # Get image and annotations for this index
        img_id = self.imgs[idx]
        img_info = self.images[img_id]
        img_path = os.path.join(self.root, img_info['file_name'])
        image = cv2.imread(img_path)  # Load image using OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = torch.from_numpy(image).to(torch.float32)  # Convert NumPy array to PyTorch tensor
        image = image.permute(2, 0, 1)  # Change the order of dimensions from (H, W, C) to (C, H, W)
        image = image / 255.0  # Normalize pixel values to [0, 1]

        # Get bounding boxes and labels
        boxes = []
        labels = []
        for ann in self.img_to_anns[img_id]:
            bbox = ann['bbox']
            boxes.append([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])  # Convert to [x_min, y_min, x_max, y_max]
            labels.append(ann['category_id'])

        # Convert boxes and labels to tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}

        return image, target

    def __len__(self):
        return len(self.imgs)


- Load Train & Validation Dataset

In [ ]:
# Assume dataset and DataLoader for training and validation have been set up
DATASET_ROOT_PATH = dataset.location

dataset_train = CustomDataset(root=f"{dataset.location}/train")
dataset_val = CustomDataset(root=f"{dataset.location}/valid")

data_loader_train = DataLoader(dataset_train, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
data_loader_val = DataLoader(dataset_val, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

print(f"Train Dataset : {len(dataset_train)} data, \t Validation Dataset : {len(dataset_val)} data")


- Set Dataset Class Labels and Count

In [ ]:
def get_dataset_count(root) :
  with open(os.path.join(root, "_annotations.coco.json")) as f:
      data = json.load(f)

  # Extract all category names into a list
  labels = [category["name"] for category in data.get("categories", [])]
  return len(labels), labels


# Define Number of dataset class and class labels based on downloaded Roboflow Dataset
NUM_CLASS, CLASS_LABELS = get_dataset_count(root=f"{dataset.location}/train")

print(f"Number of Class : {NUM_CLASS}, \t Labels : {CLASS_LABELS}")

- Load Pretrained <font color="orange">Faster R-CNN</font> Model with <font color="orange">ResNet-50</font> Backbone from [Torchvision](https://pytorch.org/vision/main/models/generated/torchvision.models.detection.fasterrcnn_resnet50_fpn_v2.html)
  - Modify number of detection class on pretrained <font color="orange">Faster R-CNN Predictor</font> (a.k.a `FastRCNNPredictor()`)

In [ ]:
# Load the pre-trained model
model = fasterrcnn_resnet50_fpn_v2(weights='DEFAULT')

# Get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASS)

# Move model to device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

- Run <font color="orange">Training Loop</font> for Pretrained Faster R-CNN with Custom Dataset downloaded from Roboflow
  - Calculate <font color="orange">Loss</font> and <font color="orange">mAP</font> on each Epoch
  - Run training loop at least for 20 epoch

In [ ]:
# Initialize mAP metric
map_metric = MeanAveragePrecision(iou_thresholds=[0.5, 0.75, 0.9],  # IoU thresholds
                                  class_metrics=True)  # Separate mAP per class


# Define SGD Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)


# Define Evaluation function to compute mAP on Evaulation dataset
def evaluate(model, validation_loader, device):
    model.eval()  # Ensure model is in eval mode

    for images, targets in validation_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.no_grad():
            outputs = model(images)

        # Format predictions and targets for torchmetrics
        preds = [
            {
                "boxes": output["boxes"].cpu(),
                "scores": output["scores"].cpu(),
                "labels": output["labels"].cpu(),
            }
            for output in outputs
        ]

        gts = [
            {
                "boxes": target["boxes"].cpu(),
                "labels": target["labels"].cpu(),
            }
            for target in targets
        ]

        # Update mAP metric with predictions and ground truths
        map_metric.update(preds, gts)

    # Compute mAP at the end of the epoch
    map_result = map_metric.compute()
    return map_result




# Training loop
NUM_EPOCH = 20
for epoch in range(NUM_EPOCH):
    model.train()

    total_loss = 0
    for images, targets in data_loader_train:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        total_loss += losses.item()

        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    # Validation step
    mAP = evaluate(model, data_loader_val, device)
    print(f"Epoch {epoch+1}, \tLoss: {total_loss/len(data_loader_train)}, \tmAP: {mAP['map']:.4f}, \tmAP-50: {mAP['map_50']:.4f}")


- Export trained faster R-CNN model into <font color="orange">ONNX Format</font>

In [ ]:
MODEL_NAME = "fasterrcnn_resnet50_fpn_v2_scissors.onnx"

torch.onnx.export(
    model,
    torch.rand(1, 3, 224, 224).to(device),
    MODEL_NAME,  # Output path
    export_params=True,  # Store trained weights
    opset_version=11,    # ONNX opset version
    do_constant_folding=True,  # Optimize model by folding constants
    input_names=["input"],  # Name of the input layer
    output_names=["boxes", "labels", "scores"],  # Names of output layers
    dynamic_axes={
        "input": {0: "batch_size"},  # Variable batch size
        "boxes": {0: "num_boxes"},   # Variable number of boxes
        "labels": {0: "num_boxes"},
        "scores": {0: "num_boxes"}
    }
)

- Download Trained Faster R-CNN ONNX Model into disk.

In [ ]:
from google.colab import files

files.download(MODEL_NAME)